<a href="https://colab.research.google.com/github/HandersThe/Capstone_Project_ML/blob/master/ProjectCapstone_rev_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0

In [ ]:
#Install kaggle
!pip install -q kaggle

#Upload kaggle api
from google.colab import files
files.upload()

In [ ]:
#Create directory for kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Download dataset and unzip
!kaggle datasets download -d techsash/waste-classification-data
!unzip waste-classification-data.zip

In [ ]:
#The number of output classes
NUM_CLASSES = 2

#Directory path
train_path = "dataset/DATASET/TRAIN/"
valid_path = "dataset/DATASET/TEST/"

#Training epochs
epochs = 50

#Directory path for saving model
model_save_location = "Model/EfficientNetV2"

In [ ]:
#Augmentation
img_augmentation = tf.keras.models.Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

#Build model function
def build_model(NUM_CLASSES):
    inputs = layers.Input(shape=(224, 224, 3))
    x = img_augmentation(inputs)

    #Using the imported version of EfficientNet
    model = EfficientNetV2B0(include_top=False, input_tensor=x, weights="imagenet")

    #Freeze the pretrained weights
    model.trainable = False

    #Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="linear", name="pred")(x)

    #Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = "adam"
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(
        optimizer=optimizer, loss=loss, metrics=["accuracy"]
    )
    return model

def unfreeze_model(model):
  #Unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(
          optimizer=optimizer, loss=loss, metrics=["accuracy"]
    )

In [ ]:
#Callback
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self,epoch,logs={}):
        if((logs.get('accuracy')>0.95) and (logs.get('val_accuracy')>0.93)):
          self.model.stop_training = True

In [ ]:
model = build_model(NUM_CLASSES)
unfreeze_model(model)

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=32)

_ = model.fit(train_batches, epochs=epochs, validation_data=valid_batches, verbose=2, callbacks=myCallback())

In [ ]:
#For testing (need additional test set)
test_path = ""

#You can try testing againts validation set by changing the directory to valid_path
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

test_labels = test_batches.classes
print("Test Labels",test_labels)
print(test_batches.class_indices)

predictions = model.predict(test_batches,steps=len(test_batches),verbose=0)

acc = 0
for i in range(len(test_labels)):
    actual_class = test_labels[i]
    if predictions[i][actual_class] > 0.5 : 
        acc += 1
print("Accuracy:",(acc/len(test_labels))*100,"%")

In [ ]:
#Save the model
model.save(model_save_location)

In [ ]:
#Converting to TF Lite
converter = tf.lite.TFLiteConverter.from_saved_model(model_save_location)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

#Writing TF Lite model
with open("model.tflite", "wb") as f:
    f.write(tflite_quant_model)

In [ ]:
#Download Model
!zip -r Model.zip /content/Model/EfficientNetV2